# Vision Transformer (ViT) for MNIST

In this notebook, we test a ViT implementation with MNIST dataset.

In [2]:
import torch
import torchvision
#
import numpy as np

## Loading the MNIST dataset

In [5]:
torch.manual_seed(42)

DOWNLOAD_PATH = "./data/mnist"
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 1000


transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize([0.1307,], [0.3081,])])

# Dataset
train_set = torchvision.datasets.MNIST(DOWNLOAD_PATH, train=True, download=True,
                                       transform=transform)
test_set  = torchvision.datasets.MNIST(DOWNLOAD_PATH, train=False, download=True,
                                       transform=transform)
# Dataloader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_set,  batch_size=BATCH_SIZE_TEST,  shuffle=True)